### 📘 Lesson 5: Market Model

<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Course presenters</h3>
    <ul>
      <li><strong>Priyesh Gosai</strong> - Energy Systems Modeler and Training Coordinator</li>
      <li><strong>Dr. Fabian Hofmann</strong> - Senior Optimization and Energy System Modelling Expert</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


#### 🎯 Learning Objectives  



* Run a demonstration model using real data. 
* Solve the Day ahead and intra day model. 
* Analyse the results.

---

#### 📈 Market Model

In [1]:
import pypsa
import pandas as pd
import numpy as np
from training_scripts import *

In [2]:
input_file_name = 'data/Lesson5_model.xlsx'
path = convert_selected_sheets_to_csv(input_file_name, 'lesson5_csv_folder')

INFO:root:Converted snapshots to CSV.
INFO:root:Converted buses to CSV.
INFO:root:Converted carriers to CSV.
INFO:root:Converted generators to CSV.
INFO:root:Converted generators-p_max_pu to CSV.
INFO:root:Converted links to CSV.
INFO:root:Converted loads to CSV.
INFO:root:Converted loads-p_set to CSV.
INFO:root:Converted storage_units to CSV.
INFO:root:Converted storage_units-inflow to CSV.
INFO:root:Conversion complete. CSV files are saved in 'lesson5_csv_folder'
INFO:root:Excel file closed successfully.


In [3]:
network = pypsa.Network()
network.import_from_csv_folder(path)

INFO:pypsa.io:Imported network lesson5_csv_folder has buses, carriers, generators, links, loads, storage_units


💧 **Setting Water Availability for Hydropower Plants**


⚡ **Modeling Demand Side Management (DSM)**  





In [4]:
network.optimize(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 6/6 [00:00<00:00, 176.47it/s]
INFO:linopy.io: Writing time: 0.35s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 10560 primals, 39200 duals
Objective: 5.72e+08
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down, Link-fix-p-lower, Link-fix-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


('ok', 'optimal')

In [5]:
del network.model

network_id = network.copy()


```


```

In [6]:
# Define the carriers that should be committable
committable_carriers = ["coal", "gas", "biomass"]

# Step 1: Identify generators that produced power during the simulation
active_generators = network.generators_t.p.sum() > 0  # Sum over time and check if nonzero

# Step 2: Filter generators that belong to the selected carriers and were active
active_and_committable = network.generators.index[network.generators["carrier"].isin(committable_carriers) & active_generators]

# Step 3: Set 'committable' = True only for those generators
network.generators.loc[active_and_committable, "committable"] = True

###
---